In [111]:
import pandas as pd
import numpy as np

In [112]:
df = pd.read_csv('pronghorn_300.csv')

In [113]:
print 'df = {}'.format(len(df))

df = 300


In [114]:
df.head()

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
0,22,2015-01-31,967.0,2015-04-20,2017-03-07,5,5,0,225000.0,NaN
1,29,2015-02-07,1292.0,2015-03-31,2018-08-22,16,18,3,4389500.0,1000000.0
2,47,2015-02-13,1189.0,2015-02-28,2017-10-03,4,5,1,182000.0,100000.0
3,51,2015-02-16,854.0,2015-03-09,2017-06-19,2,2,0,303031.0,NaN
4,83,2015-03-01,1280.0,2015-03-01,2018-08-29,81,93,10,3858346.0,1200000.0


In [115]:
dataframe = df.copy()

In [116]:
dataframe = dataframe.drop(['join_date','lama_join_in_day', 'first_trx_date', 'total_month'] , 1)

In [117]:
dataframe.isnull().sum()

uid                    0
last_trx_date          0
total_trx_success      0
total_trx_deposit      0
total_amount           0
total_deposit        259
dtype: int64

In [118]:
dataframe.total_deposit = dataframe.total_deposit.fillna(0)

In [119]:
dataframe.dtypes

uid                    int64
last_trx_date         object
total_trx_success      int64
total_trx_deposit      int64
total_amount         float64
total_deposit        float64
dtype: object

In [120]:
dataframe.last_trx_date = pd.to_datetime(dataframe['last_trx_date'])

In [121]:
dataframe.dtypes

uid                           int64
last_trx_date        datetime64[ns]
total_trx_success             int64
total_trx_deposit             int64
total_amount                float64
total_deposit               float64
dtype: object

In [122]:
date_max = dataframe.last_trx_date.max()
date_max

Timestamp('2018-08-29 00:00:00')

## RFM Score 

### RFM good costumer : LOW recency, HIGH frequency and HIGH monetary

RECENCY   : Days since last purchase, 
FREQUENCY : Total number of purchases, 
MONETARY  : Total money this customer spent

In [123]:
import datetime as dt

date_now = dt.datetime(2018,8,30)
date_now

datetime.datetime(2018, 8, 30, 0, 0)

In [124]:
dataframe['recency'] = date_now - dataframe['last_trx_date']
dataframe['recency'] = dataframe['recency'].dt.days

In [125]:
dataframe['frequency'] = dataframe['total_trx_success'] + dataframe['total_trx_deposit']

In [126]:
dataframe['monetary'] = dataframe['total_amount'] + dataframe['total_deposit']

In [127]:
dataframe.head()

,uid,last_trx_date,total_trx_success,total_trx_deposit,total_amount,total_deposit,recency,frequency,monetary
0,22,2017-03-07,5,0,225000.0,0.0,541,5,225000.0
1,29,2018-08-22,18,3,4389500.0,1000000.0,8,21,5389500.0
2,47,2017-10-03,5,1,182000.0,100000.0,331,6,282000.0
3,51,2017-06-19,2,0,303031.0,0.0,437,2,303031.0
4,83,2018-08-29,93,10,3858346.0,1200000.0,1,103,5058346.0


In [128]:
rfm_data = dataframe.copy()

In [129]:
rfm_data = rfm_data.drop(['last_trx_date', 'total_trx_success', 'total_trx_deposit', 'total_amount', 'total_deposit'] , 1)

In [130]:
rfm_data.head()

,uid,recency,frequency,monetary
0,22,541,5,225000.0
1,29,8,21,5389500.0
2,47,331,6,282000.0
3,51,437,2,303031.0
4,83,1,103,5058346.0


In [131]:
rfm_data.describe()

,uid,recency,frequency,monetary
count,300.000000,300.000000,300.000000,3.000000e+02
mean,751.506667,374.916667,18.256667,2.069749e+06
std,339.059238,215.012350,36.572978,5.932881e+06
min,22.000000,1.000000,1.000000,7.000000e+03
25%,484.000000,219.750000,2.000000,1.750000e+05
50%,765.000000,384.000000,7.000000,5.537500e+05
75%,1050.750000,563.000000,20.000000,1.740125e+06
max,1319.000000,735.000000,347.000000,7.398067e+07


In [132]:
q_vals = rfm_data.quantile(q=[0.25,0.5,0.75])
q_vals = q_vals.to_dict()

In [133]:
q_vals

{'frequency': {0.25: 2.0, 0.5: 7.0, 0.75: 20.0},
 'monetary': {0.25: 175000.0, 0.5: 553750.0, 0.75: 1740125.0},
 'recency': {0.25: 219.75, 0.5: 384.0, 0.75: 563.0},
 'uid': {0.25: 484.0, 0.5: 765.0, 0.75: 1050.75}}

In [134]:
def LowRecency(x,p,q_vals):
    if x <= q_vals[p][0.25]:
        return 4
    elif x <= q_vals[p][0.50]:
        return 3
    elif x <= q_vals[p][0.75]: 
        return 2
    else:
        return 1

def HighFM(x,p,q_vals):
    if x <= q_vals[p][0.25]:
        return 1
    elif x <= q_vals[p][0.50]:
        return 2
    elif x <= q_vals[p][0.75]: 
        return 3
    else:
        return 4

In [135]:
rfm_data['R_Quartile'] = rfm_data['recency'].apply(LowRecency, args=('recency',q_vals,))
rfm_data['F_Quartile'] = rfm_data['frequency'].apply(HighFM, args=('frequency',q_vals,))
rfm_data['M_Quartile'] = rfm_data['monetary'].apply(HighFM, args=('monetary',q_vals,))

In [136]:
rfm_data.head()

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile
0,22,541,5,225000.0,2,2,2
1,29,8,21,5389500.0,4,4,4
2,47,331,6,282000.0,3,2,2
3,51,437,2,303031.0,2,1,2
4,83,1,103,5058346.0,4,4,4


In [137]:
rfm_data ['RFM_Score'] = rfm_data.R_Quartile.map(str) + rfm_data.F_Quartile.map(str) + rfm_data.M_Quartile.map(str)

In [138]:
rfm_data.head()

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score
0,22,541,5,225000.0,2,2,2,222
1,29,8,21,5389500.0,4,4,4,444
2,47,331,6,282000.0,3,2,2,322
3,51,437,2,303031.0,2,1,2,212
4,83,1,103,5058346.0,4,4,4,444


In [139]:
rfm_data[rfm_data['RFM_Score']=='444'].sort_values('monetary', ascending=False).head(5)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score
260,1178,2,300,47289500.0,4,4,4,444
122,646,1,347,28238523.0,4,4,4,444
239,1091,6,80,24496400.0,4,4,4,444
144,739,10,204,18693267.0,4,4,4,444
41,388,3,69,14959247.0,4,4,4,444


## Labelling the customer

### Good Customer

In [140]:
rfm_data[rfm_data['RFM_Score']=='444'].sort_values('monetary', ascending=False).head(5)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score
260,1178,2,300,47289500.0,4,4,4,444
122,646,1,347,28238523.0,4,4,4,444
239,1091,6,80,24496400.0,4,4,4,444
144,739,10,204,18693267.0,4,4,4,444
41,388,3,69,14959247.0,4,4,4,444


In [141]:
df.loc[df['uid'] == 1178]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
260,1178,2015-04-23,1228.0,2015-04-23,2018-08-28,173,247,53,26768500.0,20521000.0


In [142]:
rfm_data ['Total_RFM'] = rfm_data.R_Quartile + rfm_data.F_Quartile + rfm_data.M_Quartile

In [143]:
rfm_data.head()

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Total_RFM
0,22,541,5,225000.0,2,2,2,222,6
1,29,8,21,5389500.0,4,4,4,444,12
2,47,331,6,282000.0,3,2,2,322,7
3,51,437,2,303031.0,2,1,2,212,5
4,83,1,103,5058346.0,4,4,4,444,12


### By default, the customers with 'RFM_srore' = 444 or 'Total_RFM' = 12 are potential cutomer for lending.

### Customer with 'Total_RFM' = 11 ???

In [145]:
rfm_data[rfm_data['Total_RFM'] == 11].sort_values('monetary', ascending=False).head(5)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Total_RFM
162,793,268,83,73980669.0,3,4,4,344,11
130,688,352,53,12909804.0,3,4,4,344,11
231,1070,226,100,9036278.0,3,4,4,344,11
172,833,297,92,7082000.0,3,4,4,344,11
153,774,168,14,6756260.0,4,3,4,434,11


In [146]:
df.loc[df['uid'] == 774]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
153,774,2015-04-04,1190.0,2015-04-04,2018-03-15,10,14,0,6756260.0,NaN


Note: 774 --> this customer is a potential customer, low recency with high frequency and monetary.

In [148]:
df.loc[df['uid'] == 833]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
172,833,2015-04-07,1064.0,2015-04-07,2017-11-06,70,82,10,4482000.0,2600000.0


Note : 833 --> this customer might be not a potential customer coz has high recency means haven't done a transaction in long time

Conclusion: the customer with 'Total_RFM' = 11 can be considered to be a potential customer as long as the recency value is low

### Customer with 'Total_RFM' = 10 ???

In [85]:
rfm_data[rfm_data['Total_RFM'] == 10].sort_values('monetary', ascending=False).head(5)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Total_RFM
31,350,314,13,18363757.0,3,3,4,334,10
22,280,350,19,8337500.0,3,3,4,334,10
69,453,491,50,4375000.0,2,4,4,244,10
212,1012,196,7,4059411.0,4,2,4,424,10
263,1191,402,46,3646500.0,2,4,4,244,10


In [150]:
df.loc[df['uid'] == 350]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
31,350,2015-03-31,934.0,2017-02-04,2017-10-20,9,9,4,12963757.0,5400000.0


Note : 350 --> Might be not a potential customer

In [151]:
df.loc[df['uid'] == 453]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
69,453,2015-04-02,755.0,2015-04-03,2017-04-26,42,45,5,3375000.0,1000000.0


Note : 453 --> Might be not a potential customer

In [152]:
df.loc[df['uid'] == 1012]

,uid,join_date,lama_join_in_day,first_trx_date,last_trx_date,total_month,total_trx_success,total_trx_deposit,total_amount,total_deposit
212,1012,2015-04-14,1161.0,2015-04-14,2018-02-15,5,7,0,4059411.0,NaN


Note : 1012 --> Might be not a potential customer, low frequency

In [156]:
def label_cust(row):
    if row['Total_RFM'] == 12:
        label = 'Good'
    elif row['Total_RFM'] == 11 and row['R_Quartile'] == 4:
        label = 'Good'
    else:
        label = 'Bad'
    return label

In [157]:
rfm_data['label'] = rfm_data.apply(label_cust, axis=1)

In [158]:
rfm_data.head(10)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Total_RFM,label
0,22,541,5,225000.0,2,2,2,222,6,Bad
1,29,8,21,5389500.0,4,4,4,444,12,Good
2,47,331,6,282000.0,3,2,2,322,7,Bad
3,51,437,2,303031.0,2,1,2,212,5,Bad
4,83,1,103,5058346.0,4,4,4,444,12,Good
5,86,572,1,440995.0,1,1,2,112,4,Bad
6,87,503,24,2627000.0,2,4,4,244,10,Bad
7,90,291,20,1650000.0,3,3,3,333,9,Bad
8,91,298,37,3620000.0,3,4,4,344,11,Bad
9,97,16,81,10502004.0,4,4,4,444,12,Good


In [160]:
rfm_data.loc[rfm_data['Total_RFM'] >= 11].head(10)

,uid,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFM_Score,Total_RFM,label
1,29,8,21,5389500.0,4,4,4,444,12,Good
4,83,1,103,5058346.0,4,4,4,444,12,Good
8,91,298,37,3620000.0,3,4,4,344,11,Bad
9,97,16,81,10502004.0,4,4,4,444,12,Good
12,112,30,61,4812000.0,4,4,4,444,12,Good
16,141,28,122,10244500.0,4,4,4,444,12,Good
19,226,126,231,13021250.0,4,4,4,444,12,Good
21,275,57,17,2250000.0,4,3,4,434,11,Good
27,305,2,53,4774803.0,4,4,4,444,12,Good
41,388,3,69,14959247.0,4,4,4,444,12,Good


In [161]:
rfm_data['label'].value_counts()

Bad     251
Good     49
Name: label, dtype: int64

### Result:

From 300 customers, 49 customers are protential customer for lending based on their RFM score

------------------------------------------------------------------------------------